In [2]:
import requests
import pandas as pd
from config import ykey
import numpy as np
from matplotlib import pyplot as plt
import json
import csv

## import data from step1 and calculate ratingscore

In [9]:
path='C:/Users/rwang/Desktop/Mentos/cleandata.csv'
file=pd.read_csv(path)
cleandata=pd.DataFrame(file)
ratingscore=cleandata['rating']/cleandata["review"]
cleandata["ratingscore"]=ratingscore
cleandata.sort_values(by="ratingscore",ascending=False)

,Unnamed: 0,id,review,category,rating,city,lat,lon,ratingscore
1373,1375,mRMgI_BaJg5L0SfewMEGnQ,1,seafood,5.0,Lansing,42.694698,-84.603952,5.000000
1613,1615,La7BQxKy046Ym42U2lKGSQ,1,newamerican,5.0,Orange,38.251820,-78.113865,5.000000
1775,1778,yMs_AC_twFfgFV6LyUPteA,1,japanese,5.0,Lacey,47.037478,-122.819501,5.000000
1857,1860,#NAME?,1,pizza,5.0,Pierre,44.368426,-100.356769,5.000000
1859,1862,3Y3zo3t0_sJwYBg5km4UCg,1,hotdogs,5.0,Pierre,44.369905,-100.360019,5.000000
...,...,...,...,...,...,...,...,...,...
403,404,y2w6rFaO0XEiG5mFfOsiFA,5022,seafood,4.5,Boston,42.363271,-71.056058,0.000896
229,229,bZiIIUcpgxh8mpKMDhdqbA,4602,burgers,4.0,Austin,30.249649,-97.749907,0.000869
1633,1635,GXFMD0Z4jEVZBCsbPf4CTQ,5425,tradamerican,4.5,Nashville,36.151387,-86.796603,0.000829
1101,1102,ojq84p8G6Pg0NccdDdN1jA,6110,hawaiian,4.0,Honolulu,21.276659,-157.827799,0.000655


## Calculate outlier review count

In [19]:
quartiles=cleandata["review"].quantile([.25,.5,.75])
lower=quartiles[0.25]
upper=quartiles[0.75]
iqr=upper-lower
lowerbound=lower-1.5*iqr
upperbound=upper+1.5*iqr
outlier= cleandata.loc[(cleandata['review'] < lowerbound) | (cleandata['review'] > upperbound)]
print(f"The min potential outliers review counts is {outlier['review'].min()}")

The min potential outliers review counts is 926


## Calculate outlier ratingscore

In [28]:
quartiles_rs=cleandata["ratingscore"].quantile([.25,.5,.75])
lower_rs=quartiles_rs[0.25]
upper_rs=quartiles_rs[0.75]
iqr_rs=upper_rs-lower_rs
lowerbound_rs=lower_rs-1.5*iqr_rs
upperbound_rs=upper_rs+1.5*iqr_rs
outlier_rs= cleandata.loc[(cleandata['ratingscore'] < lowerbound_rs) | (cleandata['ratingscore'] > upperbound_rs)]
print(f"The min potential outliers ratingscore is {outlier_rs['ratingscore'].min()}")

The min potential outliers ratingscore is 0.13636363636363635


#### Get analysis data

In [80]:
data1=cleandata[cleandata["ratingscore"]<0.13636363636363635]
analysisdata=data1[data1["review"]<926]
analysisdata.to_csv("analysisdata")

#### Top 10 category based off store count

In [81]:
catedata=analysisdata.groupby(analysisdata["category"])
storecount=catedata.count()
storecount.sort_values(by="id",ascending=False)["id"].head(10)

category
newamerican     264
italian         172
tradamerican    111
seafood          97
mexican          80
steak            68
bars             64
pizza            61
thai             51
southern         45
Name: id, dtype: int64

### Not suprise to see in US , American food restaurant prevales on the store number count

#### Top 10 category based off average review count by store

In [82]:
sum=catedata.sum()
avgreview=round(sum/storecount,0)
reviewsummary=avgreview.sort_values(by="review",ascending=False)
reviewsummary["review"].head(10)

category
sicilian         861.0
conveyorsushi    837.0
gluten_free      788.0
newmexican       698.0
comedyclubs      576.0
butcher          427.0
irish_pubs       421.0
latin            399.0
brasseries       399.0
tex-mex          386.0
Name: review, dtype: float64

### Mexican appears on the two top 10 list , might be somthing intersting to explore

#### Looking into avg ratingscore for mexican restaurant 

In [83]:
avg=catedata.mean()
ratingsummary=avg.sort_values(by="ratingscore",ascending=False)
ratingsummary["ratingscore"].describe()

count    128.000000
mean       0.034401
std        0.016781
min        0.005226
25%        0.024636
50%        0.031508
75%        0.042340
max        0.111111
Name: ratingscore, dtype: float64

In [84]:
ratingsummary["ratingscore"].head(35)

category
mongolian         0.111111
musicvenues       0.077922
cheesesteaks      0.073611
polish            0.070839
seafoodmarkets    0.067640
brewpubs          0.065609
colombian         0.064286
venues            0.063636
beergardens       0.063622
szechuan          0.063187
pakistani         0.061644
chinese           0.059177
beer_and_wine     0.057118
soulfood          0.057083
coffee            0.056693
malaysian         0.055556
moroccan          0.052329
lounges           0.051263
sportsbars        0.050956
gourmet           0.049166
comfortfood       0.048521
caribbean         0.048292
chickenshop       0.047821
irish             0.046530
salad             0.046432
british           0.046275
bakeries          0.046052
shanghainese      0.045455
tapas             0.045330
portuguese        0.045305
french            0.044008
vietnamese        0.042681
japanese          0.042227
diners            0.041645
mexican           0.041029
Name: ratingscore, dtype: float64

#### Mexican restaurant is among the top 30%, rating score wise

## Gather data for all mexican related restaurant

In [87]:
mexidata1=analysisdata[analysisdata["category"]=="newmexican"]
mexidata2=analysisdata[analysisdata["category"]=="mexican"]
mexidata=mexidata1.append(mexidata2).drop_duplicates(subset="city",keep='first')
mexidata.sort_values(by="ratingscore",ascending=False)
mexidata.to_csv("mexidata")

In [92]:
mexidata.describe()

,Unnamed: 0,review,rating,lat,lon,ratingscore
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,1241.024390,243.219512,4.182927,38.744173,-98.150123,0.035035
std,695.610397,215.296251,0.399313,6.083377,19.314004,0.029377
min,28.000000,36.000000,3.000000,21.352410,-157.857870,0.005597
25%,733.000000,98.000000,4.000000,34.033780,-112.073926,0.014423
50%,1171.000000,154.000000,4.000000,39.037119,-93.634530,0.027950
75%,1798.000000,312.000000,4.500000,41.768546,-84.288810,0.044554
max,2378.000000,804.000000,5.000000,58.301245,-70.892691,0.125000


## Get the best rated mexican stores

In [100]:
goodmexidata1=mexidata[mexidata["ratingscore"]>0.027950]
goodmexidata=goodmexidata1[goodmexidata1["rating"]>4]
goodmexidata.to_csv("goodmexidata")